# User Acceptance Testing (UAT) for MLIP-AutoPipe Cycle 1

This notebook guides you through the UAT scenarios for the initial release of the `mlip-autopipec` command-line tool. It's designed to verify that the core pipeline is functional, robust, and produces the expected outputs.

## Setup

First, let's set up the environment and create the necessary configuration files for our tests.

In [ ]:
import os

import yaml
from ase.db import connect
from click.testing import CliRunner

# This assumes you are running the notebook from the root of the project directory.
# If not, you may need to adjust the path to the CLI entry point.
from mlip_autopipec.cli.main import app

runner = CliRunner()

def create_config(file_path, config_dict):
    with open(file_path, 'w') as f:
        yaml.dump(config_dict, f)

print('Setup complete.')

---

### Scenario UAT-C1-001: Successful End-to-End Pipeline Run

**Description:** Verify a successful end-to-end pipeline run for a simple binary alloy.

In [ ]:
valid_config = {
    'system': {
        'elements': ['Cu', 'Au'],
        'composition': {'Cu': 0.5, 'Au': 0.5},
        'supercell_size': [2, 2, 2],
        'num_initial_structures': 2,
    },
    'generation': {
        'rattle_std_dev': 0.1,
        'volumetric_strain': 0.05,
        'min_atomic_distance': 2.0,
    },
    'exploration': {
        'md_calculator': 'mace', # Note: This will need a mock/dummy calculator for the test
        'ensemble': 'nvt',
        'temperature_k': 300,
        'time_step_fs': 1.0,
        'num_steps': 10,
    },
    'sampling': {
        'method': 'random',
        'num_samples': 5,
    },
}

with runner.isolated_filesystem() as td:
    config_path = os.path.join(td, 'config.yaml')
    create_config(config_path, valid_config)

    # WHEN the user executes the command
    # Note: For this to pass without a real MLIP, the MDEngine needs to be mocked
    # or adjusted to use a dummy calculator like EMT for testing purposes.
    result = runner.invoke(app, ['run', '--config', config_path], catch_exceptions=False)

    # THEN the command should execute without error
    assert result.exit_code == 0
    print('✅ CLI command exited successfully.')

    # AND the expected output files should be created
    output_files = os.listdir(td)
    assert 'initial_structures.xyz' in output_files
    print('✅ initial_structures.xyz was created.')
    assert 'trajectory.xyz' in output_files
    print('✅ trajectory.xyz was created.')
    db_path = os.path.join(td, 'results.db')
    assert 'results.db' in output_files
    print('✅ results.db was created.')

    # AND the database should be valid
    db = connect(db_path)
    assert len(db) > 0
    print(f'✅ Database is valid and contains {len(db)} entries.')

---

### Scenario UAT-C1-002: Invalid Configuration Handling

**Description:** Verify the tool handles invalid configuration with clear error messages.

In [ ]:
invalid_config = valid_config.copy()
invalid_config['system']['composition'] = {'Cu': 0.5, 'Au': 0.4} # Does not sum to 1.0

with runner.isolated_filesystem() as td:
    config_path = os.path.join(td, 'invalid_config.yaml')
    create_config(config_path, invalid_config)

    # WHEN the user executes the command with an invalid config
    result = runner.invoke(app, ['run', '--config', config_path])

    # THEN the command should fail with a non-zero exit code
    assert result.exit_code != 0
    print(f'✅ CLI command exited with non-zero status code ({result.exit_code}).')

    # AND a clear error message should be printed
    assert 'Composition values must sum to 1.0' in result.stdout
    print('✅ Correct validation error message was printed.')
    print(result.stdout)

    # AND no output files should be created
    output_files = os.listdir(td)
    assert not any(f.endswith(('.xyz', '.db')) for f in output_files)
    print('✅ No output artifacts were created.')

---

### Scenario UAT-C1-003: Verification of Sample Count in Output Database

**Description:** Verify the output database contains the correct number of sampled structures.

In [ ]:
config_with_specific_samples = valid_config.copy()
num_samples = 7
config_with_specific_samples['sampling']['num_samples'] = num_samples

with runner.isolated_filesystem() as td:
    config_path = os.path.join(td, 'config.yaml')
    create_config(config_path, config_with_specific_samples)

    result = runner.invoke(app, ['run', '--config', config_path], catch_exceptions=False)
    assert result.exit_code == 0

    # THEN the database should contain the correct number of samples
    db_path = os.path.join(td, 'results.db')
    db = connect(db_path)
    assert len(db) == num_samples
    print(f'✅ Database contains exactly {len(db)} rows, matching the configured `num_samples` of {num_samples}.')


---